Elige el clasificador más adecuado de entre los vistos en clase y usa scikit-learn junto con las librerías que
necesites para resolver las siguientes cuestiones. Muestra todos los resultados del algoritmo paso a paso.
1) Realiza todo el preprocesamiento necesario para poder entrenar el clasificador. (1 punto)
2) Crea un clasificador e indica su error. Úsalo para saber a qué tipo corresponden al menos 5 audios/canciones que
no hayas usado para entrenar el modelo. (1 punto) 